#### Importando as bibliotecas

In [1]:
import pandas as pd
import xlrd
import pickle #modulo usado para serializar dados para o arquivo binario
import os
import time
import copy

#### Criando as funções necessárias para acessar as árvores TRIE

In [2]:
#Definindo classe de nodos TRIE
class TrieNode(object):
    def __init__(self, char: str):   # __init__ é um método especial para fazer construtores
        self.char = char   # caractere do nodo atual
        self.filhos = []   # nodos filhos
        self.pFinalizada = False   # se é o último nodo e a palavra terminou
        self.indices = []   # lista vazia para nodos que não são término de palavra
        ### não é um índice único e sim uma lista, pois podem existir títulos de música repetidos com mais de um artista
        
#Definindo função que adiciona uma string nova a arvore
def addString(raiz, palavra: str, indice: int):   # função usada para adicionar uma palavra nova à estrutura trie

    nodo = raiz
    for char in palavra:
        encontradoEmFilho = False
        # busca pelo caractere nos filhos do nodo atual
        for filho in nodo.filhos:
            if filho.char == char: 
                nodo = filho   # apontamos o nodo para o filho que contém esse char
                encontradoEmFilho = True
                break
        
        if not encontradoEmFilho:   # se o caractere não foi encontrado, adiciona novo filho
            novoNodo = TrieNode(char)
            nodo.filhos.append(novoNodo)
            nodo = novoNodo   # apontamos, então, o nodo para seu novo filho e continuamos a iteração
    
    nodo.pFinalizada = True   # indica que até ali pode ser uma palavra (nome de música/artista completo)
    nodo.indices.append(indice)   # assinala o indice passado
    
#Defiindo função que localiza uma string na arvore
def findString(raiz, palavra: str) -> (bool, []):
    """
      1. Se a palavra existe e tem algum índice associado, retorna verdadeiro e a lista de índices dela
      2. Se a palavra não existe, retorna falso e uma lista vazia
    """
    nodo = raiz

    if not raiz.filhos:   # se o nodo raiz não tiver nenhum filho, trivial, retorna falso
        return False, []
    for char in palavra:
        charNaoEncontrado = True
        for filho in nodo.filhos:
            if filho.char == char:
                charNaoEncontrado = False   # assinala que o char foi encontrado
                nodo = filho   # passa iteração para o nodo filho
                break
        
        if charNaoEncontrado:   # retorna falso quando não encontrar algum char da palavra sendo procurada
            return False, []
    
    # Caso passe por todos os caracteres sem retornar [False, []], então significa que a palavra foi encontrada
    # Resta saber se aquele nodo é um nodo final com um índice associado
    if nodo.pFinalizada:
        return True, nodo.indices
    else:
        return False, []

#### Fazendo a leitura dos arquivos

In [3]:
#Faz a leitura do arquivo de registros
with (open('teste.bin', 'rb')) as openfile:
    dados = pickle.load(openfile)
    
#Leitura do arquivo de indices de anos
with (open('indices.bin', 'rb')) as openfile:
    indices = pickle.load(openfile)

#Leitura do arquivo com a arvore TRIE de titulos de musicas
with open('trieMusicas.bin','rb') as openfile:
    trieMusicas = pickle.load(openfile)
        
#Leitura do arquivo com a arvore TRIE de nomes de artistas
with open('trieArtistas.bin','rb') as openfile:
    trieArtistas = pickle.load(openfile)

## Definição das Querys

#### Query que retorna um numero N de pesquisas em ordem de relevancia, filtrando por artista/ano opcionalmente
###### Opcional: Depois criar um algoritmo que tente achar se tem mais de uma musica do mesmo artista em um ano

In [4]:
def relevancia(top, ano=None, artista=None):
    
    #Se nao especifica nenhum ano então retorna os mais populares no geral
    
    #Retornando de um determinado ano
    if(ano != None):
        #Consultando do arquivo de indices qual o index inicial do ano que o usuario entrou
        for x in indices:
            if x['Ano'] == ano:
                comeco = x['Min']
        for i in range(comeco, comeco+top):
            print(dados[i])
            
    #Retornando de um determinado artista
    if(artista != None):
        indicesArtista = findString(trieArtistas, artista)[1] #retorna os indices do arquivo principal com registros do artista
        listaMusicas2 = []
        for i in indicesArtista:
            listaMusicas2.append(dados[i])

        listaMusicas = copy.deepcopy(listaMusicas2) #Faz uma copia de lista2 para lista, pq lista2 so referencia os registros de dados ao inves de criar eles novamente
        
        for i in range(len(listaMusicas)-1):
            for j in range(i+1, len(listaMusicas)-1):
                #print(j)
                if(listaMusicas[i]['Titulo'] == listaMusicas[j]['Titulo']):
                    listaMusicas[i]['Pontos'] = listaMusicas[i]['Pontos'] + listaMusicas[j]['Pontos']
                    listaMusicas.pop(j)
        
        listaMusicas = sorted(listaMusicas, key = lambda tup: (tup["Pontos"]), reverse=True)
        
        if(len(listaMusicas) < top):
            top = len(listaMusicas) - 1
            
        for i in range(top):
            print(listaMusicas[i])
        
        

In [6]:
relevancia(100, 2010)

{'Artista': 'Train', 'Titulo': 'Hey, Soul Sister', 'Ano': 2010, 'Pontos': 3623, 'Peak': 98, 'Semanas': 54}
{'Artista': 'Lady Antebellum', 'Titulo': 'Need You Now', 'Ano': 2010, 'Pontos': 3485, 'Peak': 99, 'Semanas': 60}
{'Artista': 'The Script', 'Titulo': 'Breakeven', 'Ano': 2010, 'Pontos': 2679, 'Peak': 89, 'Semanas': 43}
{'Artista': 'B.o.B Featuring Hayley Williams', 'Titulo': 'Airplanes', 'Ano': 2010, 'Pontos': 2662, 'Peak': 99, 'Semanas': 30}
{'Artista': 'Usher Featuring will.i.am', 'Titulo': 'OMG', 'Ano': 2010, 'Pontos': 2620, 'Peak': 100, 'Semanas': 30}
{'Artista': 'Jason Derulo', 'Titulo': 'In My Head', 'Ano': 2010, 'Pontos': 2592, 'Peak': 96, 'Semanas': 34}
{'Artista': 'Taio Cruz', 'Titulo': 'Dynamite', 'Ano': 2010, 'Pontos': 2545, 'Peak': 99, 'Semanas': 47}
{'Artista': 'Taio Cruz Featuring Ludacris', 'Titulo': 'Break Your Heart', 'Ano': 2010, 'Pontos': 2507, 'Peak': 100, 'Semanas': 29}
{'Artista': 'Enrique Iglesias Featuring Pitbull', 'Titulo': 'I Like It', 'Ano': 2010, 'Ponto